# Credit Card Fraud

Develop an algorithm to predict fraud. Prioritize correctly finding fraud rather than correctly labeling non-fraudulent transactions.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.naive_bayes import BernoulliNB
from sklearn import linear_model
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

raw_data = pd.read_csv('./data/creditcard.csv')
raw_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Data Exploration

In [2]:
frauds = raw_data[raw_data['Class'] == 1]
okays = raw_data[raw_data['Class'] == 0]

baseline = 1 - len(frauds) / (len(frauds) + len(okays))
print('Frauds:', len(frauds))
print('Not Frauds:', len(okays))
print('Baseline R-squared:', baseline)

Frauds: 492
Not Frauds: 284315
Baseline R-squared: 0.9982725143693799


In [3]:
# plt.figure(figsize=(20,10))

# plt.subplot(2, 2, 1)
# plt.title('Fraud Amounts')
# plt.hist(frauds['Amount'], bins=20, label='Fraud', color='blue')

# plt.subplot(2, 2, 2)
# plt.title('Okay Amounts')
# plt.hist(okays['Amount'], bins=20, label='Okay', color='orange')

# plt.subplot(2, 2, 3)
# plt.title('Fraud Times')
# plt.hist(frauds['Time'], bins=20, label='Fraud', color='blue')

# plt.subplot(2, 2, 4)
# plt.title('Okay Times')
# plt.hist(okays['Time'], bins=20, label='Okay', color='orange')

# plt.show()

In [4]:
# # TODO: 
# # Try taking out outliers – see distribution of remaining amounts
# # Compare model with / without outliers

# plt.figure(figsize=(20,10))

# plt.subplot(2, 2, 1)
# plt.title('Fraud Amounts')
# plt.boxplot(frauds['Amount'])

# plt.subplot(2, 2, 2)
# plt.title('Okay Amounts')
# plt.boxplot(okays['Amount'])

# plt.subplot(2, 2, 3)
# plt.title('Fraud Times')
# plt.boxplot(frauds['Time'])

# plt.subplot(2, 2, 4)
# plt.title('Okay Times')
# plt.boxplot(okays['Time'])

# plt.show()

In [5]:
# plt.figure(figsize=(20,60))

# for i in range(28):
#     col_name = 'V' + str(i + 1)
    
#     plt.subplot(28, 2, 2 * i + 1)
#     plt.title('Frauds ' + col_name)
#     plt.hist(frauds[col_name], bins=20, color='blue')
    
#     plt.subplot(28, 2, 2 * i + 2)
#     plt.title('Okays ' + col_name)
#     plt.hist(okays[col_name], bins=20, color='orange')
    
# plt.show()

In [6]:
# plt.figure(figsize=(20,90))

# for i in range(28):
#     col_name = 'V' + str(i + 1)
    
#     plt.subplot(28, 2, 2 * i + 1)
#     plt.title('Frauds ' + col_name)
#     plt.boxplot(frauds[col_name])
    
#     plt.subplot(28, 2, 2 * i + 2)
#     plt.title('Okays ' + col_name)
#     plt.boxplot(okays[col_name])
    
# plt.show()

I am looking to minimize Type II errors ("false negative"s – i.e. considering something that is a fraud as okay). But before I begin to build models, I need to make sure my data is not skewed by either undersampling my majority group (okays) or oversampling my minority group (frauds). 

## Helper Methods

In [7]:
# Helper method for fitting and training model, returns formatted result
def fit_and_train(model, fit_X_train, fit_Y_train, X_train, Y_train):
    model_fit = model.fit(fit_X_train, fit_Y_train)
    model_score_train = model.score(X_train, Y_train)
    print('R² for train:', model_score_train)
    
    model_score_test = model.score(X_test, Y_test)
    print('\nR² for test:', model_score_test)
    
    model_improve_over_baseline = (model_score_test - baseline) / baseline
    print('Improvement over baseline:', model_improve_over_baseline)
    
#     if hasattr(model_fit, 'coef_'):
#         print('\nCoefficients:', model_fit.coef_)
    
#     if hasattr(model_fit, 'intercept_'):
#         print('\nIntercept:', model_fit.intercept_)
    
#     if hasattr(X_train, 'columns'):
#         print('Data cols:', list(X_train.columns))

# Helper method for evaluating model, returns formatted result
def evaluate_model_printout(model):
    Y_train_vals = df_train['Class'].values
    Y_test_vals = df_test['Class'].values

    predict_train = model.predict_proba(X_train)
    predict_train = list(map(lambda x: 0 if x[0] > .998 else 1, predict_train))
    predict_train = np.fromiter(predict_train, dtype=np.int)

    predict_test = model.predict_proba(X_test)
    predict_test = list(map(lambda x: 0 if x[0] > .998 else 1, predict_test))
    predict_test = np.fromiter(predict_test, dtype=np.int)
    
    crosstab_labels = [0, 1, 'All']
    table_train = pd.crosstab(Y_train_vals, predict_train, rownames=['actual'], colnames=['predicted'], margins=True)
    table_train = table_train.reindex(index=crosstab_labels,columns=crosstab_labels, fill_value=0)

    print('TRAIN:')
#     print(table_train, '\n')

    train_tI_errors = table_train.loc[0,1] / table_train.loc['All','All']
    train_tII_errors = table_train.loc[1,0] / table_train.loc['All','All']
    print(('Accuracy:\n% Type I errors: {}\n% Type II errors: {}\n').format(train_tI_errors, train_tII_errors))

    train_precision = table_train.loc[1,1] / table_train.loc['All', 1] # correctly predicted positives / all predicted positives
    train_recall = table_train.loc[1,1] / table_train.loc[1,'All'] # true positives / (true positives + false negatives)
    print('Precision:', train_precision)
    print('Recall:', train_recall, '\n\n----------\n')

    table_test = pd.crosstab(Y_test_vals, predict_test, rownames=['actual'], colnames=['predicted'], margins=True)
    table_test = table_test.reindex(index=crosstab_labels,columns=crosstab_labels, fill_value=0)
    
    print('TEST:')
#     print(table_test, '\n')

    test_tI_errors = table_test.loc[0,1]/table_test.loc['All','All']
    test_tII_errors = table_test.loc[1,0]/table_test.loc['All','All']
    print(('Accuracy:\n% Type I errors: {}\n% Type II errors: {}\n').format(test_tI_errors, test_tII_errors))

    test_precision = table_test.loc[1,1] / table_test.loc['All', 1] # correctly predicted positives / all predicted positives
    test_recall = table_test.loc[1,1] / table_test.loc[1,'All'] # true positives / (true positives + false negatives)
    print('Precision:', test_precision)
    print('Recall:', test_recall)

## STRATEGY 1: Undersampling Okays

In [8]:
print('Randomly sample', len(frauds), '(# of frauds) rows from okays (3 times)')
all_undersampled_okays = []

for time in range(3):
    rand_sample = okays.sample(n=len(frauds))
    all_undersampled_okays.append(rand_sample)

for idx, sample in enumerate(all_undersampled_okays):
    df = pd.concat([frauds, sample])
    df = df.reset_index()
    df_test = df.iloc[::2]
    df_train = df.iloc[1::2]

    # TRAINING
    X_train = df_train.loc[:, ~(df_train.columns).isin(['Class', 'index', 'Time', 'Amount'])]
    Y_train = df_train['Class'].values.reshape(-1, 1)

    # TESTING
    X_test = df_test.loc[:, ~(df_train.columns).isin(['Class', 'index', 'Time', 'Amount'])]
    Y_test = df_test['Class'].values.reshape(-1, 1)

#     # LASSO
#     lasso = linear_model.LogisticRegression(penalty='l1', C=100) 
#     fit_and_train(lasso, X_train, Y_train, X_train, Y_train)
#     evaluate_model_printout(lasso)
    
#     # RIDGE
#     ridge = linear_model.LogisticRegression(penalty='l2', C=100, fit_intercept=False)
#     fit_and_train(ridge, X_train, Y_train, X_train, Y_train)
#     evaluate_model_printout(ridge)
    
#     # SVC
#     svm = SVC(kernel='linear', probability=True)
#     fit_and_train(svm, X_train, Y_train, X_train, Y_train)

#     # Naive Bayes
#     bnb = BernoulliNB()
#     fit_and_train(bnb, X_train, Y_train, X_train, Y_train)

    print('*** GRADIENT BOOSTING CLASSIFIER', idx + 1, '***')
    gbm = ensemble.GradientBoostingClassifier(n_estimators=500, max_depth=2, loss='deviance')
    fit_and_train(gbm, X_train, Y_train, X_train, Y_train)
    evaluate_model_printout(gbm)
    
    print('*** RANDOM FOREST', idx + 1, '***')
    rfc = ensemble.RandomForestClassifier()
    fit_and_train(rfc, X_train, Y_train, X_train, Y_train)
    evaluate_model_printout(rfc)

Randomly sample 492 (# of frauds) rows from okays (3 times)
*** GRADIENT BOOSTING CLASSIFIER 1 ***


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R² for train: 1.0

R² for test: 0.9329268292682927
Improvement over baseline: -0.06545876417559868
TRAIN:
Accuracy:
% Type I errors: 0.2032520325203252
% Type II errors: 0.0

Precision: 0.7109826589595376
Recall: 1.0 

----------

TEST:
predicted    0    1  All
actual                  
0          115  131  246
1            1  245  246
All        116  376  492 

Accuracy:
% Type I errors: 0.266260162601626
% Type II errors: 0.0020325203252032522

Precision: 0.651595744680851
Recall: 0.9959349593495935
*** RANDOM FOREST 1 ***
R² for train: 0.9817073170731707

R² for test: 0.9349593495934959
Improvement over baseline: -0.06342272662478306
TRAIN:
Accuracy:
% Type I errors: 0.15853658536585366
% Type II errors: 0.0

Precision: 0.7592592592592593
Recall: 1.0 

----------

TEST:
predicted    0    1  All
actual                  
0          108  138  246
1            4  242  246
All        112  380  492 

Accuracy:
% Type I errors: 0.2804878048780488
% Type II errors: 0.008130081300813009

Prec

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R² for train: 1.0

R² for test: 0.9451219512195121
Improvement over baseline: -0.05324253887070464
TRAIN:
Accuracy:
% Type I errors: 0.18089430894308944
% Type II errors: 0.0

Precision: 0.7343283582089553
Recall: 1.0 

----------

TEST:
predicted    0    1  All
actual                  
0          136  110  246
1            1  245  246
All        137  355  492 

Accuracy:
% Type I errors: 0.22357723577235772
% Type II errors: 0.0020325203252032522

Precision: 0.6901408450704225
Recall: 0.9959349593495935
*** RANDOM FOREST 2 ***
R² for train: 0.991869918699187

R² for test: 0.9410569105691057
Improvement over baseline: -0.05731461397233599
TRAIN:
Accuracy:
% Type I errors: 0.18089430894308944
% Type II errors: 0.0

Precision: 0.7343283582089553
Recall: 1.0 

----------

TEST:
predicted    0    1  All
actual                  
0          102  144  246
1            2  244  246
All        104  388  492 

Accuracy:
% Type I errors: 0.2926829268292683
% Type II errors: 0.0040650406504065045



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R² for train: 1.0

R² for test: 0.9349593495934959
Improvement over baseline: -0.06342272662478306
TRAIN:
Accuracy:
% Type I errors: 0.16463414634146342
% Type II errors: 0.0

Precision: 0.7522935779816514
Recall: 1.0 

----------

TEST:
predicted    0    1  All
actual                  
0          125  121  246
1            0  246  246
All        125  367  492 

Accuracy:
% Type I errors: 0.2459349593495935
% Type II errors: 0.0

Precision: 0.670299727520436
Recall: 1.0
*** RANDOM FOREST 3 ***
R² for train: 0.9939024390243902

R² for test: 0.9532520325203252
Improvement over baseline: -0.04509838866744184
TRAIN:
Accuracy:
% Type I errors: 0.16666666666666666
% Type II errors: 0.0

Precision: 0.75
Recall: 1.0 

----------

TEST:
predicted   0    1  All
actual                 
0          97  149  246
1           2  244  246
All        99  393  492 

Accuracy:
% Type I errors: 0.30284552845528456
% Type II errors: 0.0040650406504065045

Precision: 0.6208651399491094
Recall: 0.991869918699

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


I am looking to minimize Type II errors ("false negative"s – i.e. considering something that is a fraud as okay). My test set gave 0.0% for Type II errors, which I am very happy with, especially since my percentage of Type I errors is also extremely low (~0.000035%).

In [9]:
# NOTE: Could use ROC / AUC threshold for less performant classifiers

In [10]:
# PCA on models that work to reduce 
# Find out if multicollinearity using LASSO – remove features to improve

## STRATEGY 2: Over-Sampling Frauds

In [11]:
df_test = raw_data.iloc[::2]
df_train = raw_data.iloc[1::2]

X_train = df_train.loc[:, ~(df_train.columns).isin(['Class'])]
Y_train = df_train['Class'].values.reshape(-1, 1)

# TESTING
X_test = df_test.loc[:, ~(df_train.columns).isin(['Class'])]
Y_test = df_test['Class'].values.reshape(-1, 1)

sm = SMOTE(random_state=12, ratio = 1.0)
X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### LASSO Logistic Regression

In [12]:
lasso = linear_model.LogisticRegression(penalty='l1', C=100) 
fit_and_train(lasso, X_train_res, Y_train_res, X_train, Y_train)

R² for train: 0.992120952508023

R² for test: 0.9922544310553074
Improvement over baseline: -0.006028497456803355


#### Evaluating LASSO Logistic Regression

In [13]:
evaluate_model_printout(lasso)

TRAIN:
Accuracy:
% Type I errors: 0.8685069837011861
% Type II errors: 0.0

Precision: 0.0019448187929211823
Recall: 1.0 

----------

TEST:
predicted      0       1     All
actual                          
0          18496  123657  142153
1              1     250     251
All        18497  123907  142404 

Accuracy:
% Type I errors: 0.8683534170388473
% Type II errors: 7.022274655206314e-06

Precision: 0.0020176422639560316
Recall: 0.9960159362549801


### Ridge Logistic Regression

In [14]:
ridge = linear_model.LogisticRegression(penalty='l2', C=100, fit_intercept=False)
fit_and_train(ridge, X_train_res, Y_train_res, X_train, Y_train)

R² for train: 0.9842348826920782

R² for test: 0.9841366815538889
Improvement over baseline: -0.014160294520801055


#### Evaluating Ridge Logistic Regression

In [15]:
evaluate_model_printout(ridge)

TRAIN:
Accuracy:
% Type I errors: 0.9979845930212145
% Type II errors: 0.0

Precision: 0.0016929269372071623
Recall: 1.0 

----------

TEST:
predicted   0       1     All
actual                       
0          61  142092  142153
1           0     251     251
All        61  142343  142404 

Accuracy:
% Type I errors: 0.9978090503075756
% Type II errors: 0.0

Precision: 0.0017633462832735013
Recall: 1.0


### Gradient Boosting Classifier

In [16]:
gbm = ensemble.GradientBoostingClassifier(n_estimators=500, max_depth=2, loss='deviance')
fit_and_train(gbm, X_train_res, Y_train_res, X_train, Y_train)

KeyboardInterrupt: 

#### Evaluating Gradient Boosting Classifier

In [ ]:
evaluate_model_printout(gbm)

### <s>Support Vector Machine</s>

In [ ]:
# svm = SVC(kernel='linear', probability=True)
# fit_and_train(svm, X_train_res, Y_train_res, X_train, Y_train)

#### Evaluating Support Vector Machine

In [ ]:
# evaluate_model_printout(svm)

### Random Forest Classifier

In [ ]:
rfc = ensemble.RandomForestClassifier()
fit_and_train(rfc, X_train_res, Y_train_res, X_train, Y_train)

#### Evaluating Random Forest Classifier

In [ ]:
evaluate_model_printout(rfc)

### Naive Bayes

In [ ]:
bnb = BernoulliNB()
fit_and_train(bnb, X_train_res, Y_train_res, X_train, Y_train)

#### Evaluating Naive Bayes

In [ ]:
evaluate_model_printout(bnb)

## Conclusion

Under-sampling the majority class (i.e. not frauds) proved a better strategy to deal with the skewed dataset than over-sampling the minority class (i.e. frauds), as the models built after under-sampling were much more accurate.

Of all the classifiers built using under-sampling, the Naive Bayes model was the most performant, as it 1) minimized errors, especially Type II errors (Type I - 1~5%, Type II - ~2%), and 2) showed the highest precision (~75%) and recall (~95%) rate. For a problem like this, we are more concerned with a high recall rate than precision rate.

In [ ]:
# MENTOR NOTES:
# Precision – % of my + predictions that are correct
# Recall – % of my target +s I predicted

# CURRICULUM NOTES:
# There are a few things you can do to deal with class imbalance:

# Ignore. If we really only care about the absolute accuracy of the model and our sample is representative of the population, 
# this can be a reasonable strategy. Engineer features that strongly identify the minority class, and this can turn out ok.

# Change your sampling. If you oversample the minority class or undersample the majority class, you can create a more balanced training set. 
# This is particularly useful if the goal of your model is to correctly identify the minority class. This can also be done by creating synthetic samples 
# to try to make your data more balanced or weighting samples to balance out your classes. 

# Probability outputs. 
# Although Naive Bayes' probability outputs are generally inaccurate, other models will give you a more accurate probability of a certain class. 
# e.g. logistic regression or support vector machines (SVM)
# Instead of just taking the most likely outcome, you can set up a specific cutoff or a more complex rule. 
# In the binary case, it could be going with the minority case if it has a priority greater than some threshold.

# Lastly, you can create cost functions for errors. This quantifies ways in which errors are not equal – scale the cost of an error up or down. 
# This can mean something like a Type II error being twice as bad as a Type I error, or however you choose to quantify that relationship. 
# SKLearn's Naive Bayes model does not have an easy built-in way to do this, but it's a good thing to keep in mind.